<a href="https://colab.research.google.com/github/gurer1hsn/Sanatturleri/blob/main/BERT_ve_T5_Kar%C5%9F%C4%B1la%C5%9Ft%C4%B1rmas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers: Hugging Face modelleri için
# Datasets: Hazır veri setlerini kolayca yüklemek için
# scikit-learn: Accuracy ve F1-score gibi metrikler için
!pip install transformers datasets scikit-learn -q


In [2]:
import torch
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    T5Tokenizer, T5ForConditionalGeneration,
    Trainer, TrainingArguments,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt

# GPU varsa onu kullanıyoruz, yoksa CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)


Kullanılan cihaz: cuda


In [3]:
!pip install datasets --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_

In [4]:
from datasets import load_dataset

try:
    imdb = load_dataset("imdb")
    print("imdb dataset loaded successfully.")
except ValueError as e:
    if "Invalid pattern" in str(e):
        print("Error loading imdb dataset:", e)
        print("Please ensure the file pattern in the datasets library is correct.")
    else:
        raise e


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Error loading imdb dataset: Invalid pattern: '**' can only be an entire path component
Please ensure the file pattern in the datasets library is correct.


In [8]:
from datasets import load_dataset

try:
    imdb = load_dataset("imdb")  # imdb veri setini yüklemeye çalış
    print("imdb veri seti başarıyla yüklendi.")
    # ✅ imdb değişkenini try bloğu içinde kullan
    imdb_train = imdb["train"]  # imdb veri setinden 'train' bölümünü al
    imdb_valid = imdb["test"]  # 'test' bölümünü doğrulama seti olarak varsay
    imdb_test = imdb["unsupervised"]  # 'unsupervised' bölümünü test seti olarak varsay
except ValueError as e:
    if "Invalid pattern" in str(e):
        print("imdb veri setini yüklerken hata oluştu:", e)
        print("Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.")
    else:
        raise e

imdb veri setini yüklerken hata oluştu: Invalid pattern: '**' can only be an entire path component
Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.


In [9]:
# GEREKLİ KÜTÜPHANELERİ İÇE AKTARIYORUZ
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset

# BERT MODELİNİ YÜKLÜYORUZ
bert_model_name = "bert-base-uncased"
tokenizer_bert = BertTokenizer.from_pretrained(bert_model_name)
model_bert = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Her metni tokenize ediyoruz (maksimum 256 token, padding yapılıyor)
def tokenize_bert(example):
    return tokenizer_bert(example["text"], padding="max_length", truncation=True, max_length=256)


In [12]:
from datasets import load_dataset

try:
    imdb = load_dataset("imdb")  # imdb veri setini yüklemeye çalış
    print("imdb veri seti başarıyla yüklendi.")
    # ✅ imdb değişkenini try bloğu içinde kullan
    imdb_train = imdb["train"]  # imdb veri setinden 'train' bölümünü al
    imdb_valid = imdb["test"]  # 'test' bölümünü doğrulama seti olarak varsay
    imdb_test = imdb["unsupervised"]  # 'unsupervised' bölümünü test seti olarak varsay
except ValueError as e:
    if "Invalid pattern" in str(e):
        print("imdb veri setini yüklerken hata oluştu:", e)
        print("Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.")
    else:
        raise e

imdb veri setini yüklerken hata oluştu: Invalid pattern: '**' can only be an entire path component
Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.


In [13]:
# Accuracy ve F1-score hesaplayan fonksiyon
def compute_metrics(pred):
    labels = pred.label_ids                      # Gerçek etiketler
    preds = pred.predictions.argmax(-1)          # Tahmin edilen sınıf
    acc = accuracy_score(labels, preds)          # Doğruluk
    f1 = f1_score(labels, preds, average="macro")# Makro F1
    return {"accuracy": acc, "f1": f1}


In [14]:
from transformers import TrainingArguments

training_args_bert = TrainingArguments(
    output_dir="./bert-imdb",               # Modelin kaydedileceği klasör
    num_train_epochs=3,                     # Epoch sayısı
    per_device_train_batch_size=16,         # Batch size
    per_device_eval_batch_size=16,
    # evaluation_strategy="epoch",            # Her epoch sonunda doğrulama yap # Removed: This parameter is likely not supported in the current transformers version.
    # logging_strategy="epoch",               # Logları da her epoch sonunda göster # Removed: This parameter is likely not supported in the current transformers version.
    save_strategy="no",                     # Modeli her seferinde kaydetme
    report_to="none"                        # W&B gibi servislere gönderme
)

In [15]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        """
        Overriding the evaluation loop to ensure inputs are moved to the correct device.
        """
        self.model.eval()
        all_preds = []
        all_labels = []
        losses = []

        for step, inputs in enumerate(dataloader):
            # ✅ inputs sözlüğündeki her öğeyi cihaz (GPU) üzerine taşı
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            inputs = self._prepare_inputs(inputs)

            with torch.no_grad():
                outputs = self.model(**inputs)
                if "labels" in inputs:
                    loss = outputs.loss
                    losses.append(loss.item())
                    logits = outputs.logits
                    preds = torch.argmax(logits, axis=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(inputs["labels"].cpu().numpy())

        metrics = {}
        if all_labels:
            from sklearn.metrics import accuracy_score, f1_score
            metrics["accuracy"] = accuracy_score(all_labels, all_preds)
            metrics["f1"] = f1_score(all_labels, all_preds, average="weighted")

        if losses:
            metrics["eval_loss"] = sum(losses) / len(losses)

        return metrics

In [16]:
# Modeli eğitiyoruz
num_train_epochs=1  # TrainingArguments içinde değiştir



In [17]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        """
        Overriding the evaluation loop to ensure inputs are moved to the correct device.
        """
        self.model.eval()
        all_preds = []
        all_labels = []
        losses = []

        for step, inputs in enumerate(dataloader):
            # ✅ inputs sözlüğündeki her öğeyi cihaz (GPU) üzerine taşı
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            inputs = self._prepare_inputs(inputs)

            with torch.no_grad():
                outputs = self.model(**inputs)
                if "labels" in inputs:
                    loss = outputs.loss
                    losses.append(loss.item())
                    logits = outputs.logits
                    preds = torch.argmax(logits, axis=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(inputs["labels"].cpu().numpy())

        metrics = {}
        if all_labels:
            from sklearn.metrics import accuracy_score, f1_score
            metrics["accuracy"] = accuracy_score(all_labels, all_preds)
            metrics["f1"] = f1_score(all_labels, all_preds, average="weighted")

        if losses:
            metrics["eval_loss"] = sum(losses) / len(losses)

        return metrics


In [24]:
!pip install datasets -q # This will make sure the dataset is downloaded.
import torch
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    T5Tokenizer, T5ForConditionalGeneration,
    Trainer, TrainingArguments,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt

# GPU varsa onu kullanıyoruz, yoksa CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

# BERT MODELİNİ YÜKLÜYORUZ
bert_model_name = "bert-base-uncased"
tokenizer_bert = BertTokenizer.from_pretrained(bert_model_name)
model_bert = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)

# Define `tokenize_bert` here before it's called, for example:
def tokenize_bert(example):
    return tokenizer_bert(example["text"], padding="max_length", truncation=True, max_length=256)

from datasets import load_dataset

try:
    imdb = load_dataset("imdb")  # imdb veri setini yüklemeye çalış
    print("imdb veri seti başarıyla yüklendi.")
    # ✅ imdb değişkenini try bloğu içinde kullan
    imdb_train = imdb["train"]  # imdb veri setinden 'train' bölümünü al
    imdb_valid = imdb["test"]  # 'test' bölümünü doğrulama seti olarak varsay
    imdb_test = imdb["unsupervised"]  # 'unsupervised' bölümünü test seti olarak varsay
    # imdb veri setini tokenleştir # Moved this part inside the try block
    imdb_train_enc = imdb_train.map(tokenize_bert, batched=True)
    imdb_valid_enc = imdb_valid.map(tokenize_bert, batched=True)

    # PyTorch tensör formatına çevir
    imdb_train_enc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    imdb_valid_enc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
except ValueError as e:
    if "Invalid pattern" in str(e):
        print("imdb veri setini yüklerken hata oluştu:", e)
        print("Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.")
    else:
        raise e

Kullanılan cihaz: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


imdb veri setini yüklerken hata oluştu: Invalid pattern: '**' can only be an entire path component
Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.


In [25]:
t5_model_name = "t5-base"
tokenizer_t5 = T5Tokenizer.from_pretrained(t5_model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [27]:
!pip install datasets -q # This will make sure the dataset is downloaded.
import torch
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    T5Tokenizer, T5ForConditionalGeneration,
    Trainer, TrainingArguments,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt

# GPU varsa onu kullanıyoruz, yoksa CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

# BERT MODELİNİ YÜKLÜYORUZ
bert_model_name = "bert-base-uncased"
tokenizer_bert = BertTokenizer.from_pretrained(bert_model_name)
model_bert = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)


# 0 -> "olumsuz", 1 -> "olumlu", -1 -> "bilinmiyor"
label_map = {0: "negative", 1: "positive", -1: "unknown"}

# T5 için metni giriş, etiketleri string olarak hedef olarak veriyoruz
def preprocess_t5(examples):
    input_texts = examples["text"]
   # Her örnek için 'etiket'in bir liste olup olmadığını kontrol edin ve eğer öyleyse, ilk öğeyi alın
    labels = [example_label[0] if isinstance(example_label, list) else example_label
              for example_label in examples["label"]]
    # Etiketleri int'e dönüştür ve label_map'te yoksa "unknown" olarak işaretle
    target_texts = [label_map.get(int(label), "unknown") for label in labels]
    # Girdi metni tokenize edilir
    input_enc = tokenizer_t5(input_texts, max_length=256, padding="max_length", truncation=True)
    # Çıktı (label) metni tokenize edilir
    with tokenizer_t5.as_target_tokenizer():
        target_enc = tokenizer_t5(target_texts, max_length=5, padding="max_length", truncation=True)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

# Define `tokenize_bert` here before it's called, for example:
def tokenize_bert(example):
    return tokenizer_bert(example["text"], padding="max_length", truncation=True, max_length=256)

from datasets import load_dataset

try:
    imdb = load_dataset("imdb")  # imdb veri setini yüklemeye çalış
    print("imdb veri seti başarıyla yüklendi.")
    # ✅ imdb değişkenini try bloğu içinde kullan
    imdb_train = imdb["train"]  # imdb veri setinden 'train' bölümünü al
    imdb_valid = imdb["test"]  # 'test' bölümünü doğrulama seti olarak varsay
    imdb_test = imdb["unsupervised"]  # 'unsupervised' bölümünü test seti olarak varsay
    # imdb veri setini tokenleştir # Moved this part inside the try block
    imdb_train_enc = imdb_train.map(tokenize_bert, batched=True)
    imdb_valid_enc = imdb_valid.map(tokenize_bert, batched=True)

    # PyTorch tensör formatına çevir
    imdb_train_enc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    imdb_valid_enc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    # Tokenizasyonu veri kümelerine uygula # Moved this part inside the try block
    imdb_train_t5 = imdb_train.map(preprocess_t5, batched=True)
    imdb_valid_t5 = imdb_valid.map(preprocess_t5, batched=True)
    imdb_test_t5 = imdb_test.map(preprocess_t5, batched=True)
except ValueError as e:
    if "Invalid pattern" in str(e):
        print("imdb veri setini yüklerken hata oluştu:", e)
        print("Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.")
    else:
        raise e

Kullanılan cihaz: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


imdb veri setini yüklerken hata oluştu: Invalid pattern: '**' can only be an entire path component
Lütfen datasets kütüphanesindeki dosya düzeninin doğru olduğundan emin olun.


In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer_t5 = T5Tokenizer.from_pretrained("t5-base")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-base")


In [29]:
from transformers import DataCollatorForSeq2Seq # DataCollatorForSeq2Seq sınıfını içe aktar
data_collator_t5 = DataCollatorForSeq2Seq(tokenizer=tokenizer_t5, model=model_t5) # Beklenmeyen girinti kaldırıldı

In [30]:
def compute_metrics_t5(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer_t5.batch_decode(predictions, skip_special_tokens=True)
    labels[labels == -100] = tokenizer_t5.pad_token_id
    decoded_labels = tokenizer_t5.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]
    acc = accuracy_score(decoded_labels, decoded_preds)
    f1 = f1_score(decoded_labels, decoded_preds, average="macro")
    return {"accuracy": acc, "f1": f1}


In [31]:
from transformers import Seq2SeqTrainingArguments


In [32]:
from transformers import Seq2SeqTrainingArguments

training_args_t5_emotion = Seq2SeqTrainingArguments(
    output_dir="./t5-emotion",
    # "evaluation_strategy" has likely been removed or replaced.
    # Instead, use the "eval_steps" or "eval_accumulation_steps" parameters.
    # For example, to evaluate every 500 steps:
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

from transformers import Seq2SeqTrainingArguments

training_args_t5 = Seq2SeqTrainingArguments(
    output_dir="./t5-results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_strategy="no",
    report_to="none"
)


In [33]:
def preprocess_emotion_t5(example):
    input_text = example["text"]
    target_text = emotion_label_map[int(example["label"])]

    # Girdi metni tokenize edilir
    input_enc = tokenizer_t5(input_text, max_length=128, padding="max_length", truncation=True)

    # Çıktı metni tokenize edilir (etiketi metin olarak)
    with tokenizer_t5.as_target_tokenizer():
        target_enc = tokenizer_t5(target_text, max_length=5, padding="max_length", truncation=True)

    # labels'ı liste değil, tensor olarak döndür
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc


In [34]:
def preprocess_t5(examples):
    input_texts = examples["text"]
    # Her örnek için 'etiket'in bir liste olup olmadığını kontrol edin
    # ve eğer öyleyse, ilk öğeyi alın, değilse olduğu gibi bırakın
    labels = [example_label[0] if isinstance(example_label, list) else example_label
              for example_label in examples["label"]]
    target_texts = [label_map[int(label)] for label in labels] # Etiketleri int'e dönüştür
    # Girdi metni tokenize edilir
    input_enc = tokenizer_t5(input_texts, max_length=256, padding="max_length", truncation=True)
    # Çıktı (label) metni tokenize edilir
    with tokenizer_t5.as_target_tokenizer():
        target_enc = tokenizer_t5(target_texts, max_length=5, padding="max_length", truncation=True)
    # "labels" anahtarının değerini target_enc["input_ids"] olarak ayarlayın
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

In [4]:
from datasets import load_dataset

# Emotion veri kümesini Hugging Face üzerinden yükle
# 'emotion' yerine 'SetFit/emotion' kullanın
# Remove the `config_name` argument or change it to `name`
# ✅ 'multilabel' yerine 'default' yapılandırma adını kullanın
emotion = load_dataset("SetFit/emotion", name="default")  # Use name instead of config_name

# Eğitim, doğrulama ve test kümelerini alıyoruz
emotion_train = emotion["train"]
emotion_valid = emotion["validation"]
emotion_test = emotion["test"]

# Kaç örnek var kontrol edelim
print("Eğitim:", len(emotion_train))
print("Doğrulama:", len(emotion_valid))
print("Test:", len(emotion_test))

# İlk veri örneğini görelim
emotion_train[0]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/276k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/279k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Eğitim: 16000
Doğrulama: 2000
Test: 2000


{'text': 'i didnt feel humiliated', 'label': 0, 'label_text': 'sadness'}

In [5]:
# 6 sınıflı bir problem olduğu için num_labels=6 olarak tanımlıyoruz
model_bert_emotion = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)


NameError: name 'BertForSequenceClassification' is not defined

In [ ]:
# Tokenizer daha önce yüklenmişti: tokenizer_bert
def tokenize_emotion(example):
    return tokenizer_bert(example["text"], padding="max_length", truncation=True, max_length=128)


In [ ]:
# Veri kümelerine tokenizer'ı uyguluyoruz
emotion_train_enc = emotion_train.map(tokenize_emotion, batched=True)
emotion_valid_enc = emotion_valid.map(tokenize_emotion, batched=True)
emotion_test_enc = emotion_test.map(tokenize_emotion, batched=True)

# PyTorch tensör formatına çevir
for ds in [emotion_train_enc, emotion_valid_enc, emotion_test_enc]:
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [ ]:
training_args_bert_emotion = TrainingArguments(
    output_dir="./bert-emotion",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
   # evaluation_strategy ve logging_strategy'yi kaldırın
  # evaluation_strategy="epoch",
  # logging_strategy="epoch",
    save_strategy="no",
    report_to="none"
)

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import torch  # Import torch explicitly

# ... (rest of the code remains the same)

# PyTorch tensör formatına dönüştür
# Etiketlerin kelime hazinesi aralığında olduğundan emin ol
def format_dataset(dataset):
    # Etiketlerin kelime hazinesi aralığında olduğundan emin ol
    # ✅ Etiketlerin 0 ile 5 arasında olmasını sağla (model 6 sınıflı)
    # ✅ Tensor işlemleri için torch.clamp kullan
    dataset = dataset.map(lambda example: {'label': torch.clamp(example['label'], 0, 5)}, batched=True)

    # Ardından PyTorch tensör formatına dönüştür
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    return dataset

emotion_train_enc = format_dataset(emotion_train_enc)
emotion_valid_enc = format_dataset(emotion_valid_enc)
emotion_test_enc = format_dataset(emotion_test_enc)

# ... (rest of the code remains the same)

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import torch

# PyTorch tensör formatına dönüştür
def format_dataset(dataset):
    def fix_label(example):
        label = example["label"]
        if isinstance(label, list):
            label = label[0]
        label = int(label)
        label = max(0, min(5, label))  # 0 ile 5 arasında sınırla
        return {"label": label}

    dataset = dataset.map(fix_label, batched=False)
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    return dataset

emotion_train_enc = format_dataset(emotion_train_enc)
emotion_valid_enc = format_dataset(emotion_valid_enc)
emotion_test_enc = format_dataset(emotion_test_enc)

# Eğitim ayarları
training_args_bert_emotion = TrainingArguments(
    output_dir="./bert-emotion",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="no",
    report_to="none",
    eval_steps=500,
    logging_steps=500
)

# CustomTrainer sınıfı
class CustomTrainer(Trainer):
    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        self.model.eval()
        all_preds = []
        all_labels = []
        losses = []

        for step, inputs in enumerate(dataloader):
            # Move all items in the inputs dictionary to the device (GPU)
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            inputs = self._prepare_inputs(inputs)

            with torch.no_grad():
                outputs = self.model(**inputs)
                if "labels" in inputs:
                    loss = outputs.loss
                    losses.append(loss.item())
                    logits = outputs.logits
                    preds = torch.argmax(logits, axis=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(inputs["labels"].cpu().numpy())

        metrics = {}
        if all_labels:
            metrics["accuracy"] = accuracy_score(all_labels, all_preds)
            metrics["f1"] = f1_score(all_labels, all_preds, average="weighted")
        if losses:
            metrics["eval_loss"] = sum(losses) / len(losses)

        return metrics

# Move the model to the GPU/CPU within the Trainer
trainer_bert_emotion = CustomTrainer(
    # Explicitly move the model to the correct device
    model=model_bert_emotion.to(device),
    args=training_args_bert_emotion,
    train_dataset=emotion_train_enc,
    eval_dataset=emotion_valid_enc,
    compute_metrics=lambda p: {
        "accuracy": accuracy_score(p.label_ids, p.predictions.argmax(-1)),
        "f1": f1_score(p.label_ids, p.predictions.argmax(-1), average="weighted"),
    }
)

# Test verisi üzerinde değerlendirme
results = trainer_bert_emotion.evaluate(emotion_test_enc)
print(results)

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import torch  # Torch'u açıkça içe aktar

# ... (kodun geri kalanı aynı kalır)

# PyTorch tensör formatına dönüştür
# Etiketlerin kelime hazinesi aralığında olduğundan emin ol
def format_dataset(dataset):
    def fix_label(example):
        label = example["label"]
        if isinstance(label, list):
            label = label[0]  # Eğer etiket bir listeyse, ilk elemanı al
        label = int(label)  # Etiketi tam sayıya dönüştür
        label = max(0, min(5, label))  # 0 ile 5 arasında sınırla
        return {"label": label}

    dataset = dataset.map(fix_label, batched=False)  # fix_label'ı her örneğe ayrı ayrı uygula
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    return dataset


emotion_train_enc = format_dataset(emotion_train_enc)
emotion_valid_enc = format_dataset(emotion_valid_enc)
emotion_test_enc = format_dataset(emotion_test_enc)

# ... (kodun geri kalanı aynı kalır)

# Emotion BERT modeli için TrainingArguments'ı doğru eval_steps ile tanımla
training_args_bert_emotion = TrainingArguments(
    output_dir="./bert-emotion",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="no",
    report_to="none",
    eval_steps=500, # Her 500 adımda değerlendir
    logging_steps=500  # Her 500 adımda logla

)

# Emotion için BERT modelini tanımla ve eğit
class CustomTrainer(Trainer):
    # ✅ evaluation_loop metodunu CustomTrainer sınıfının içine ekleyin
    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        """
        Doğrulama döngüsünü geçersiz kılarak girdilerin doğru cihaza (GPU) taşınmasını sağlar.
        """
        self.model.eval()
        all_preds = []
        all_labels = []
        losses = []

        for step, inputs in enumerate(dataloader):
            # ✅ inputs sözlüğündeki her öğeyi cihaza (GPU) taşı
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            inputs = self._prepare_inputs(inputs)

            with torch.no_grad():
                outputs = self.model(**inputs)
                if "labels" in inputs:
                    loss = outputs.loss
                    losses.append(loss.item())
                    logits = outputs.logits
                    preds = torch.argmax(logits, axis=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(inputs["labels"].cpu().numpy())

        metrics = {}
        if all_labels:
            from sklearn.metrics import accuracy_score, f1_score
            metrics["accuracy"] = accuracy_score(all_labels, all_preds)
            metrics["f1"] = f1_score(all_labels, all_preds, average="weighted")

        if losses:
            metrics["eval_loss"] = sum(losses) / len(losses)

        return metrics

trainer_bert_emotion = CustomTrainer(
    model=model_bert_emotion,  # Emotion BERT modelini kullan
    args=training_args_bert_emotion,
    train_dataset=emotion_train_enc,
    eval_dataset=emotion_valid_enc,  # Emotion doğrulama veri setini kullan
    compute_metrics=compute_metrics
)


# Test seti üzerinde değerlendirme yap
results = trainer_bert_emotion.evaluate(emotion_test_enc)
print(results)

In [8]:
from transformers import Trainer, TrainingArguments
from transformers import BertForSequenceClassification, BertTokenizer # Import BertTokenizer here
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import torch  # Torch'u açıkça içe aktar
import pandas as pd
import matplotlib.pyplot as plt

# ... (rest of the code remains the same)

# Define the tokenizer_bert before using it
bert_model_name = "bert-base-uncased"
tokenizer_bert = BertTokenizer.from_pretrained(bert_model_name)

# ... (rest of the code remains the same)

# PyTorch tensör formatına dönüştür
# Etiketlerin kelime hazinesi aralığında olduğundan emin ol
def format_dataset(dataset):
    def fix_label(example):
        label = example["label"]
        if isinstance(label, list):
            label = label[0]  # Eğer etiket bir listeyse, ilk elemanı al
        label = int(label)  # Etiketi tam sayıya dönüştür
        label = max(0, min(5, label))  # 0 ile 5 arasında sınırla
        return {"label": label}

    dataset = dataset.map(fix_label, batched=False)  # fix_label'ı her örneğe ayrı ayrı uygula
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    return dataset


# Emotion veri kümesini Hugging Face üzerinden yükle
emotion_dataset = load_dataset("emotion")

# Eğitim, doğrulama ve test kümelerini ayır
emotion_train = emotion_dataset["train"]
emotion_valid = emotion_dataset["validation"]
emotion_test = emotion_dataset["test"]

# Tokenizer daha önce yüklenmişti: tokenizer_bert
def tokenize_emotion(example):
    return tokenizer_bert(example["text"], padding="max_length", truncation=True, max_length=128)

# Veri kümelerine tokenizer'ı uyguluyoruz
emotion_train_enc = emotion_train.map(tokenize_emotion, batched=True)
emotion_valid_enc = emotion_valid.map(tokenize_emotion, batched=True)
emotion_test_enc = emotion_test.map(tokenize_emotion, batched=True)

emotion_train_enc = format_dataset(emotion_train_enc)
emotion_valid_enc = format_dataset(emotion_valid_enc)
emotion_test_enc = format_dataset(emotion_test_enc)

# ... (rest of the code remains the same)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
# Label mapping: sayısal -> string
emotion_label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Tokenizasyon fonksiyonu (girdi metni + hedef etiket string olarak)
def preprocess_emotion_t5(example):
    input_text = example["text"]
    target_text = emotion_label_map[int(example["label"])]
    # Girdi metni tokenizasyonu
    input_enc = tokenizer_t5(input_text, max_length=128, padding="max_length", truncation=True)
    # Çıktı metni tokenizasyonu
    with tokenizer_t5.as_target_tokenizer():
        target_enc = tokenizer_t5(target_text, max_length=5, padding="max_length", truncation=True)
    input


In [10]:
from datasets import load_dataset

# Emotion veri kümesini Hugging Face üzerinden yükle
emotion_dataset = load_dataset("emotion")

# Eğitim, doğrulama ve test kümelerini ayır
emotion_train = emotion_dataset["train"]
emotion_valid = emotion_dataset["validation"]
emotion_test = emotion_dataset["test"]


In [11]:
from transformers import TrainingArguments

training_args_bert_emotion = TrainingArguments(
    output_dir="./bert-emotion",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Remove evaluation_strategy and logging_strategy
    # evaluation_strategy="epoch",
    # logging_strategy="epoch",
    save_strategy="no",
    report_to="none",
    # Add evaluation_strategy and logging_strategy using new parameters
    eval_steps=500,  # Evaluate every 500 steps
    logging_steps=500  # Log every 500 steps

)

In [12]:
# Other imports...
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, T5Tokenizer, T5ForConditionalGeneration  # Import T5Tokenizer and T5ForConditionalGeneration

# Initialize the T5 tokenizer and model
t5_model_name = "t5-base"
tokenizer_t5 = T5Tokenizer.from_pretrained(t5_model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name)

# Label mapping: sayısal -> string
emotion_label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Tokenizasyon fonksiyonu (girdi metni + hedef etiket string olarak)
def preprocess_emotion_t5(example):
    input_text = example["text"]
    target_text = emotion_label_map[int(example["label"])]
    # Girdi metni tokenizasyonu
    input_enc = tokenizer_t5(input_text, max_length=128, padding="max_length", truncation=True)
    # Çıktı metni tokenizasyonu
    with tokenizer_t5.as_target_tokenizer():
        target_enc = tokenizer_t5(target_text, max_length=5, padding="max_length", truncation=True)
    # "labels"

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
# Label mapping: sayısal -> string
emotion_label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Tokenizasyon fonksiyonu (girdi metni + hedef etiket string olarak)
def preprocess_emotion_t5(examples):
    input_texts = examples["text"]
    labels = [example_label[0] if isinstance(example_label, list) else example_label
              for example_label in examples["label"]]
    target_texts = [emotion_label_map[int(label)] for label in labels]

    input_encodings = tokenizer_t5(input_texts, max_length=128, padding="max_length", truncation=True)

    with tokenizer_t5.as_target_tokenizer():
        target_encodings = tokenizer_t5(target_texts, max_length=5, padding="max_length", truncation=True)

    # Ensure 'labels' is always a list of lists (even for single examples)
    input_encodings["labels"] = [target_encoding for target_encoding in target_encodings["input_ids"]]

    return input_encodings

In [14]:
from datasets import load_dataset

# Emotion veri kümesini Hugging Face üzerinden yükle
emotion_dataset = load_dataset("emotion")

# Eğitim, doğrulama ve test kümelerini ayır
emotion_train = emotion_dataset["train"]
emotion_valid = emotion_dataset["validation"]
emotion_test = emotion_dataset["test"]

In [15]:
from transformers import Seq2SeqTrainingArguments

training_args_t5_emotion = Seq2SeqTrainingArguments(
    output_dir="./t5-emotion",
    # "evaluation_strategy" has been replaced with "evaluation_strategy"
    # evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    logging_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="none",
    # Use eval_accumulation_steps instead of evaluation_strategy
    eval_accumulation_steps=1 # evaluate at the end of each epoch
)

In [16]:
# Other imports...
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, T5Tokenizer, T5ForConditionalGeneration  # Import T5Tokenizer and T5ForConditionalGeneration

# Initialize the T5 tokenizer and model
t5_model_name = "t5-base"
tokenizer_t5 = T5Tokenizer.from_pretrained(t5_model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name)

# Label mapping: sayısal -> string
emotion_label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# ... (your existing preprocess_emotion_t5 function) ...

# ... (your emotion_dataset loading code) ...

# ... (your training_args_t5_emotion definition) ...

# Define data collator
data_collator_t5_emotion = DataCollatorForSeq2Seq(tokenizer=tokenizer_t5, model=model_t5)

def preprocess_emotion_t5(examples):
    input_texts = examples["text"]
    labels = [emotion_label_map[int(label)] for label in examples["label"]]

    # Girdi metni tokenizasyonu
    model_inputs = tokenizer_t5(
        input_texts, max_length=128, padding="max_length", truncation=True
    )

    # Hedef metni (etiketler) tokenizasyonu
    with tokenizer_t5.as_target_tokenizer():
        tokenized_targets = tokenizer_t5(
            labels,
            max_length=5,
            padding="max_length",
            truncation=True
        )

    # 'labels' alanı liste içinde liste olmalı
    # The error is because the 'labels' should contain a list of token IDs for each example,
    # rather than a single list of token IDs.
    # We use a list comprehension to fix this:
    # model_inputs["labels"] = [ids for ids in tokenized_targets["input_ids"]]
    # model_inputs["labels"] = tokenized_targets["input_ids"]  # Assign the list of token IDs directly
    # ✅ 'labels' anahtarına her örnek için token ID'lerinin bir listesini içeren bir liste atayın.
    model_inputs["labels"] = [ids for ids in tokenized_targets["input_ids"]]  # Wrap token IDs in a list for each example

    return model_inputs

    # Trainer için eğitim argümanları
training_args_t5_emotion = Seq2SeqTrainingArguments(
    output_dir="./t5-emotion",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="none",
    eval_steps=500,
    logging_steps=500
)

# Trainer nesnesini tanımla
trainer_t5_emotion = Seq2SeqTrainer(
    model=model_t5,
    args=training_args_t5_emotion,
    train_dataset=emotion_train.map(preprocess_emotion_t5, batched=True),
    eval_dataset=emotion_valid.map(preprocess_emotion_t5, batched=True),
    data_collator=data_collator_t5_emotion
)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
import pandas as pd
import matplotlib.pyplot as plt

# Eğitim loglarını DataFrame'e çevir
log_emotion_t5 = pd.DataFrame(trainer_t5_emotion.state.log_history)

# 'eval_loss' varsa filtrele ve resetle
if 'eval_loss' in log_emotion_t5.columns:
    log_emotion_t5 = log_emotion_t5[log_emotion_t5['eval_loss'].notna()].reset_index(drop=True)

    # 'epoch' sütunu eksikse ya da boşsa manuel olarak doldur
    if 'epoch' not in log_emotion_t5.columns or log_emotion_t5['epoch'].isnull().all():
        log_emotion_t5['epoch'] = range(1, len(log_emotion_t5) + 1)

    # Eğitim kaybı ve doğruluğu grafiği
    plt.figure(figsize=(8, 5))
    plt.plot(log_emotion_t5["epoch"], log_emotion_t5["eval_loss"], label="Doğrulama Kaybı", marker='o')



